In [13]:
import sqlite3
import pandas as pd

conn = sqlite3.connect('localdata.sqlite')

In [21]:
import numpy as np
 
def convert_epsg2056_to_epsg4326(x, y):
    """
    Convert coordinates from EPSG:2056 to EPSG:4326.
 
    Args:
        x (float): The x-coordinate in EPSG:2056.
        y (float): The y-coordinate in EPSG:2056.
        z (float): The altitude in EPSG:2056.
 
    Returns:
        tuple: A tuple (lng, lat, alt) in EPSG:4326.
    """
    y_aux, x_aux = (x - 2600000) / 1e6, (y - 1200000) / 1e6
 
    lat = (16.9023892 + 3.238272 * x_aux - 0.270978 * y_aux**2 - 0.002528 * x_aux**2
           - 0.0447 * y_aux**2 * x_aux - 0.0140 * x_aux**3) * 100 / 36
 
    lng = (2.6779094 + 4.728982 * y_aux + 0.791484 * y_aux * x_aux + 0.1306 * y_aux * x_aux**2
           - 0.0436 * y_aux**3) * 100 / 36
  
    return lng, lat

In [19]:
convert_epsg2056_to_epsg4326(536788,151922 ,406)

(-14.66814349103035, 34.91024106055895, 505.27495712)

In [47]:
def plz_6_to_4_digits(plz: int) -> int:
    """
    Convert a 6-digit Swiss postal code to a 4-digit one.
 
    Args:
        plz (int): A 6-digit Swiss postal code.
 
    Returns:
        int: The corresponding 4-digit Swiss postal code.
    """
    return plz // 100

In [49]:
data = pd.read_sql_query("SELECT * FROM plz", conn)
plzs = data.loc[:,["plz_pk", "x", "y", "altitude", "primary_name"]]
plzs[["lon", "lat"]] = plzs.apply(lambda row: convert_epsg2056_to_epsg4326(row["x"]+2000000, row["y"]+1000000), axis=1, result_type="expand")
plzs["plz"] = plzs.apply(lambda row: row["plz_pk"] // 100, axis=1, result_type="expand")
plzs

,plz_pk,x,y,altitude,primary_name,lon,lat,plz
0,100003,536788,151922,406,Lausanne,6.614940,46.515614,1000
1,100025,542061,156539,837,Lausanne,6.683067,46.557622,1000
2,100101,538100,152040,448,Lausanne,6.632019,46.516798,1001
3,100200,538130,152320,490,Lausanne,6.632372,46.519320,1002
4,100300,538100,152444,488,Lausanne,6.631965,46.520432,1003
...,...,...,...,...,...,...,...,...
4266,965200,732830,232722,766,Neu St. Johann,9.192824,47.232061,9652
4267,965500,735393,229244,834,Stein SG,9.225614,47.200265,9655
4268,965600,739159,228336,887,Alt St. Johann,9.275022,47.191318,9656
4269,965700,741833,229135,914,Unterwasser,9.310547,47.197934,9657


In [31]:
data = pd.read_sql_query("SELECT * FROM wetterstation", conn)
wetterstationen = data.loc[:,["name", "x", "y", "altitude", "temperature", "sunshine", "precipitation"]]
wetterstationen[["lon", "lat"]] = wetterstationen.apply(lambda row: convert_epsg2056_to_epsg4326(row["x"]+2000000, row["y"]+1000000), axis=1, result_type="expand")

wetterstationen

,name,x,y,altitude,temperature,sunshine,precipitation,lon,lat
0,Villars-Tiercelin,544197,163650,859.0,1,0,1,6.710060,46.621768
1,Visp,631151,128023,639.0,1,1,1,7.842964,46.302890
2,La Chaux-de-Fonds,550918,214861,1017.0,1,1,1,6.792300,47.082939
3,Scuol,817139,186392,1304.0,1,1,1,10.283267,46.793263
4,"Teufen, AR",747784,250409,834.0,0,0,0,9.396061,47.387921
...,...,...,...,...,...,...,...,...,...
470,Bad Ragaz,756911,209350,497.0,1,1,1,9.502592,47.016624
471,Schleitheim,678235,288717,497.0,0,0,0,8.481762,47.744351
472,Schmerikon,713726,231532,408.0,0,0,0,8.940296,47.224924
473,Glarus,723757,210566,517.0,1,1,1,9.066967,47.034575


In [41]:
data = pd.read_sql_query("SELECT * FROM poi", conn)
pois = data.loc[:,["primary_name", "x", "y", "altitude", "type_en"]]
pois["type"] = pois["type_en"].map(lambda s: s.upper().replace(" ", "_"))
pois[["lon", "lat"]] = pois.apply(lambda row: convert_epsg2056_to_epsg4326(row["x"]+2000000, row["y"]+1000000), axis=1, result_type="expand")

pois

,primary_name,x,y,altitude,type_en,type,lon,lat
0,Cabane Arpitettaz CAS,618614,105516,2786,Mountain Hut,MOUNTAIN_HUT,7.679345,46.100892
1,Glattalphütte SAC,709410,197202,1898,Mountain Hut,MOUNTAIN_HUT,8.875022,46.916906
2,Täschhütte SAC,630293,100081,2701,Mountain Hut,MOUNTAIN_HUT,7.830020,46.051579
3,Balmhornhütte SAC,619285,144461,1956,Mountain Hut,MOUNTAIN_HUT,7.689632,46.451204
4,Chamanna Jenatsch CAS,775351,155672,2652,Mountain Hut,MOUNTAIN_HUT,9.724441,46.529356
...,...,...,...,...,...,...,...,...
618,Pizzo Ruscada,689118,114643,2004,Peak,PEAK,8.592722,46.177357
619,Piz Uccello,735107,151369,2724,Peak,PEAK,9.198783,46.499994
620,Monte San Giorgio - Mendrisio,717251,85839,1097,Peak,PEAK,8.949784,45.913993
621,Rifugio Alpe Masnee,702048,125855,2063,Mountain Hut,MOUNTAIN_HUT,8.762586,46.276366


json template

```json
{
    "weatherStations": [
        {
            "name": "Zürich",
            "measurementTypes": [
                "temperature",
                "sunshine",
                "precipitation"
            ],
            "location": {
                "latitude": 46.6333,
                "longitude": 6.7,
                "altitude": 800
            }
        }
    ],
    "pois": [
        {
            "name": "Mont Tendre",
            "type": "PEAK",
            "location": {
                "latitude": 46.6333,
                "longitude": 6.7,
                "altitude": 800
            }
        }
    ],
    "plz": [
        {
            "name": "Zürich",
            "plz": "1058",
            "location": {
                "latitude": 46.6333,
                "longitude": 6.7,
                "altitude": 800
            }
        }
    ]
}
```

In [36]:
def get_location(row):
    return {"latitude": row["lat"], "longitude": row["lon"], "altitude": row["altitude"]}

In [37]:
def get_measurement_types(row):
    return {"temperature": row["temperature"]==True, "sunshine": row["sunshine"]==True, "precipitation": row["precipitation"]==True}

In [54]:
import json

# a json is generated with the data, according to the template

data = {
    "weatherStations": wetterstationen.apply(lambda row: {"name": row["name"], "measurementTypes": get_measurement_types(row), "location": get_location(row)  }, axis=1).tolist(),
    "pois": pois.apply(lambda row: {"name": row["primary_name"], "type": row["type"], "location": get_location(row)  }, axis=1).tolist(),
    "plz": plzs.apply(lambda row: {"name": row["primary_name"], "plz": row["plz"], "location": get_location(row)  }, axis=1).tolist(),
}

with open("data.json", "w") as f:
    json.dump(data, f, ensure_ascii=False)

data["weatherStations"]

[{'name': 'Villars-Tiercelin',
  'measurementTypes': {'temperature': True,
   'sunshine': False,
   'precipitation': True},
  'location': {'latitude': 46.62176829438428,
   'longitude': 6.710059558923346,
   'altitude': '859.0'}},
 {'name': 'Visp',
  'measurementTypes': {'temperature': True,
   'sunshine': True,
   'precipitation': True},
  'location': {'latitude': 46.30288996971511,
   'longitude': 7.842964018883361,
   'altitude': '639.0'}},
 {'name': 'La Chaux-de-Fonds',
  'measurementTypes': {'temperature': True,
   'sunshine': True,
   'precipitation': True},
  'location': {'latitude': 47.08293932795008,
   'longitude': 6.792299807369331,
   'altitude': '1017.0'}},
 {'name': 'Scuol',
  'measurementTypes': {'temperature': True,
   'sunshine': True,
   'precipitation': True},
  'location': {'latitude': 46.793262795465694,
   'longitude': 10.283266666675795,
   'altitude': '1304.0'}},
 {'name': 'Teufen, AR',
  'measurementTypes': {'temperature': False,
   'sunshine': False,
   'preci